# Tutorial on calculating effective masses for Si

In [2]:
import os
import sys
import re
from siman import header
from siman.header import _update_configuration, db
from siman.database import read_database, write_database
from siman.calc_manage import smart_structure_read, add_loop, res_loop
import project_sets
from pymatgen.io.vasp import BSVasprun
from siman.set_functions import read_vasp_sets
from siman.electronic.band_structure import effective_mass as em
from siman.electronic.band_structure import band_structure as bs
from siman.wrappers import waiting_calc_complete as wrap
from siman.wrappers import sets_wrap as swrap

In [19]:
read_database() # read database if exist
_update_configuration('simanrc.py') # simanrc.py should be in the folder with project 

In [4]:
# Defining the basic parameters for calculations

sp_pack = {'KSPACING': 0.33, 'NPAR':None,'ISTART':None,'NELM':None,'PREC':None,'ALGO':None,
           'KGAMMA':None,'ENCUT':600,'LPLANE':None, 'LREAL':'.FALSE.',
           'EDIFF': 1.e-8}

mass_pack = {'LORBIT': 11, 'ICHARG': 11}

**1 Step. Perform single-point and band structure calculations**

In [6]:
# Let's use the already prepared POSCAR file and vasprun.xml file from band structure calculations (see tutorial band.py)

st_name = 'Si'
path_to_POSCAR = f'{st_name}/{st_name}.POSCAR_mass'

vasprun_band_path = "/Users/irinabrodyagina/PycharmProjects/find_k-path/Si/Si.band_file/1.vasprun.xml"
kpoints_band_path = "/Users/irinabrodyagina/PycharmProjects/find_k-path/Si/Si.band_file/KPOINTS"

siman_st = smart_structure_read(path_to_POSCAR)

**2 Step. Determine the extremes of the valence and the conduction bands and determine a set of k-points near these extremes to calculate the difference derivative**

In [9]:
# We will use pymatgen to search for extremes

v = BSVasprun(vasprun_band_path, occu_tol=1e-08)
bs = v.get_band_structure(kpoints_filename=kpoints_band_path, line_mode=True)

kpt_cbm = bs.get_cbm()['kpoint'].cart_coords # conductive band min
kpt_vbm = bs.get_vbm()['kpoint'].cart_coords # valence band max

In [10]:
# Siman can calculate both the full effective mass tensor and a single value in a certain direction in the reverse space. 
# Let's calculate the tensor for electrons (at kpt_cbm point), and for holes (at kpt_vbm point) the values in direction (1, 0, 0)

# Prepare the k-points

h = 0.05 #step between points, the recommend value about 0.05 angstreme
k_points_elec = em.generate_kpoints_for_mass(kpt_cbm, h=h, vasprun_band_path=vasprun_band_path, flag='tensor', debug=True)



Results k-points_mass in cart:
 0.972018 0.000000 0.000000  1
 1.022018 0.000000 0.000000  1
 0.922018 0.000000 0.000000  1
 0.972018 0.050000 0.000000  1
 0.972018 -0.050000 0.000000  1
 0.972018 0.000000 0.050000  1
 0.972018 0.000000 -0.050000  1
 1.022018 0.050000 0.000000  1
 0.922018 -0.050000 0.000000  1
 1.022018 -0.050000 0.000000  1
 0.922018 0.050000 0.000000  1
 1.022018 0.000000 0.050000  1
 0.922018 0.000000 -0.050000  1
 1.022018 0.000000 -0.050000  1
 0.922018 0.000000 0.050000  1
 0.972018 0.050000 0.050000  1
 0.972018 -0.050000 -0.050000  1
 0.972018 0.050000 -0.050000  1
 0.972018 -0.050000 0.050000  1


Results k-points_mass in rec:
 0.000000 0.423077 0.423077  1
 0.000000 0.444840 0.444840  1
 0.000000 0.401314 0.401314  1
 0.021763 0.423077 0.444840  1
 -0.021763 0.423077 0.401314  1
 0.021763 0.444840 0.423077  1
 -0.021763 0.401314 0.423077  1
 0.021763 0.444840 0.466603  1
 -0.021763 0.401314 0.379551  1
 -0.021763 0.444840 0.423077  1
 0.021763 0.401314 0.4

In [11]:
k_points_hole = em.generate_kpoints_for_mass(kpt_vbm, h=h, vasprun_band_path=vasprun_band_path, flag='direct', direct=(1, 0, 0), debug=True)



Results k-points_mass in cart:
 0.000000 0.000000 0.000000  1
 0.050000 0.000000 0.000000  1
 -0.050000 0.000000 0.000000  1


Results k-points_mass in rec:
 0.000000 0.000000 0.000000  1
 0.000000 0.021763 0.021763  1
 0.000000 -0.021763 -0.021763  1


In [12]:
# Create new sets for Siman

name_set_elec = "mass_elec"
name_set_hole = "mass_hole"

swrap.create_new_set(name_set_elec, "sp", mass_pack, k_path=k_points_elec, path_key='k_effective_mass')
swrap.create_new_set(name_set_hole, "sp", mass_pack, k_path=k_points_hole, path_key='k_effective_mass')


Attention! You have chosen to override set mass_elec
 


Attention! You have chosen to override set mass_hole
 



**3 Step. Performing calculations**

In [13]:
# Vasp non-self-consistent calc

band_file_name = st_name + '.if'
vasprun_mass_elec = f'{st_name}/{band_file_name}.{name_set_elec}/'
vasprun_mass_hole = f'{st_name}/{band_file_name}.{name_set_hole}/'

In [ ]:
wrap.non_self_consist_calc(st_name, name_set_elec, "sp", path_to_POSCAR=path_to_POSCAR, path_to_vasprun=vasprun_mass_elec + '1.vasprun.xml')
wrap.non_self_consist_calc(st_name, name_set_hole, "sp", path_to_POSCAR=path_to_POSCAR, path_to_vasprun=vasprun_mass_hole + '1.vasprun.xml')

In [15]:
# Tensor of effective masses for electron (band=5)

mass, eigenval_mass  = em.eff_mass_value(band=5, h=h, folder_mass=vasprun_mass_elec, vasprun_name='1.vasprun.xml', 
                                         kpt_extr=kpt_cbm, flag='tensor', debug=True)


tensor of masses
1.04468   0.00000   0.00000  
0.00000   5.06065   0.00000  
0.00000   0.00000   5.06065  

eigenvals:
 [0.957, 0.198, 0.198]

eigenvectors:

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [17]:
# Value of effective mass for holes in direction (1, 0, 0)
# Band 3, 4 - heavy hole, band 2 - light hole

for band in [4, 3, 2]:
    mass, _  = em.eff_mass_value(band=band, h=h, folder_mass=vasprun_mass_hole, vasprun_name='1.vasprun.xml',
                    kpt_extr=kpt_vbm, flag='direct', direct=(1, 0, 0), debug=True)

Effective mass in point [0. 0. 0.] for direction (1, 0, 0) = -0.274 m_e; band = 4
Effective mass in point [0. 0. 0.] for direction (1, 0, 0) = -0.274 m_e; band = 3
Effective mass in point [0. 0. 0.] for direction (1, 0, 0) = -0.162 m_e; band = 2
